# Financial Data Preprocessing

In this notebook we process financial data. 

Then we create feature space, which consists of: 1. ESG features and 2. financial features.



In [21]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from pandas_datareader import data as pdr
import yfinance as yf
import pickle
from lxml import html
from io import StringIO

In [191]:
path = '/Users/ChrisQAQ/Downloads/ICAF_2020/data1/'

### Ticker File

In [217]:
# get gvkey list txt file
gvkey_ref = pd.read_excel(path+'company_list/gvkey_ticker_reference.xlsx') 
# get through WRDS SAS studio .. from COMPA.FUNDA table

gvkey_list = gvkey_ref.gvkey.tolist()
gvkey_list = [str(key) for key in gvkey_list]
gvkey_prcd = []
for key in gvkey_list:
    n = len(key)
    if n<6:
        key = '0'*(6-n)+key
    gvkey_prcd.append(key)

gvkey_txt = ('\n').join(gvkey_prcd)
with open(path + 'company_list/scholar_data_gvkey_list.txt', 'w') as file:
    file.write(gvkey_txt)

In [218]:
# get finance features from yahoo finance
scholar_data_ticker_list = open(path+'company_list/scholar_data_ticker_list.txt').readlines()
scholar_data_ticker_list = [tic.replace('\n','') for tic in scholar_data_ticker_list]
print(f'# of unique stock tickers: {len(scholar_data_ticker_list)}')

# of unique stock tickers: 93


### Pull Data from WRDS Database

* use WRDS tools to query financial indicators

* table1: Financial Ratios Firm Level by WRDS
* table2: Compustat Daily Updates - Fundamentals Quarterly


* input: scholar_data_ticker_list.txt, scholar_data_gvkey_list.txt
* output: Financial Ratios Firm Level by WRDS.csv, Compustat Daily Updates - Fundamentals Quarterly.csv

In [228]:
ratio_raw = pd.read_csv(path+'financial/WRDS/Financial Ratios Firm Level by WRDS.csv')
funda_raw = pd.read_csv(path+'financial/WRDS/Compustat Daily Updates - Fundamentals Quarterly.csv')


### Ratio Data Prep

In [229]:
ratio_raw.head()

,gvkey,adate,qdate,public_date,pe_exi,pe_inc,ps,npm,roa,roe,de_ratio,quick_ratio,curr_ratio,ptb
0,1072,19950331.0,19950630,19951031,30.822,30.822,2.585,0.082,0.287,0.056,0.494,1.209,2.256,5.333
1,1072,19950331.0,19950930,19951130,23.566,23.566,2.263,0.094,0.316,0.129,0.472,1.317,2.382,4.296
2,1072,19950331.0,19950930,19951231,21.721,21.721,2.086,0.094,0.316,0.129,0.472,1.317,2.382,3.960
3,1072,19950331.0,19950930,19960131,21.516,21.516,2.067,0.094,0.316,0.129,0.472,1.317,2.382,3.922
4,1072,19950331.0,19951231,19960229,17.626,17.626,1.829,0.103,0.337,0.189,0.456,1.360,2.456,3.504


In [230]:
ratio_raw['month'] = ratio_raw['public_date'].apply(lambda x:str(x)[:4]+'-'+str(x)[4:6])
ratio_raw = pd.merge(ratio_raw, gvkey_to_tic[['tic','Global Company Key']], how='left',
                     left_on='gvkey', right_on='Global Company Key')
ratio_prcd = ratio_raw[['month','tic','pe_inc','ptb','ps','roa','roe','quick_ratio','curr_ratio','npm','de_ratio']]

In [231]:
ratio_prcd.head()

,month,tic,pe_inc,ptb,ps,roa,roe,quick_ratio,curr_ratio,npm,de_ratio
0,1995-10,AVX,30.822,5.333,2.585,0.287,0.056,1.209,2.256,0.082,0.494
1,1995-11,AVX,23.566,4.296,2.263,0.316,0.129,1.317,2.382,0.094,0.472
2,1995-12,AVX,21.721,3.960,2.086,0.316,0.129,1.317,2.382,0.094,0.472
3,1996-01,AVX,21.516,3.922,2.067,0.316,0.129,1.317,2.382,0.094,0.472
4,1996-02,AVX,17.626,3.504,1.829,0.337,0.189,1.360,2.456,0.103,0.456


In [232]:
print(ratio_prcd.month.min())
print(ratio_prcd.month.max())

1990-01
2019-01


### Fundamental Data Prep

In [233]:
funda_raw.head()

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,conm,ajexq,curcdq,datacqtr,datafqtr,epspiq,costat,prccq,gsector
0,1072,19900331,1990,1,INDL,C,D,STD,AVX,AVX CORP,2.0,USD,1990Q1,1990Q1,NaN,I,NaN,45
1,1072,19900630,1990,2,INDL,C,D,STD,AVX,AVX CORP,2.0,USD,1990Q2,1990Q2,NaN,I,NaN,45
2,1072,19900930,1990,3,INDL,C,D,STD,AVX,AVX CORP,2.0,USD,1990Q3,1990Q3,NaN,I,NaN,45
3,1072,19901231,1990,4,INDL,C,D,STD,AVX,AVX CORP,2.0,USD,1990Q4,1990Q4,NaN,I,NaN,45
4,1072,19910331,1991,1,INDL,C,D,STD,AVX,AVX CORP,2.0,USD,1991Q1,1991Q1,NaN,I,NaN,45


In [234]:
funda_raw['yearmonth'] = funda_raw['datadate'].apply(lambda x:str(x)[:4]+'-'+str(x)[4:6])
# turn quarter data into month data by filling the other month with the same data
funda_raw['year'] = funda_raw['yearmonth'].apply(lambda x:x[:4])
funda_raw['month'] = funda_raw['yearmonth'].apply(lambda x:int(x[-2:]))
funda_raw['monthset'] = funda_raw['month'].apply(lambda x:[((x-1)//3)*3+1,((x-1)//3)*3+2,((x-1)//3)*3+3])


In [235]:
def Get_Exd_Data(df, lst_col):
    return pd.DataFrame({
        col:np.repeat(df[col].values, df[lst_col].str.len())
        for col in df.columns.difference([lst_col])
    }).assign(**{lst_col:np.concatenate(df[lst_col].values)})[df.columns.tolist()]

exd_funda = Get_Exd_Data(funda_raw, 'monthset')
def Exd_Month_Str(i):
    if len(str(i))==1:
        return '0'+str(i)
    else:return str(i)
exd_funda['monthset'] = exd_funda['monthset'].apply(Exd_Month_Str)
exd_funda['month'] = exd_funda['year']+'-'+exd_funda['monthset']
funda_prcd = exd_funda[['month','tic','gsector','epspiq','prccq']]

In [236]:
funda_prcd.tail()

,month,tic,gsector,epspiq,prccq
26359,2019-11,EPAM,45,1.35,212.16
26360,2019-12,EPAM,45,1.35,212.16
26361,2020-01,EPAM,45,1.55,185.66
26362,2020-02,EPAM,45,1.55,185.66
26363,2020-03,EPAM,45,1.55,185.66


In [237]:
print(funda_prcd.month.min())
print(funda_prcd.month.max())

1990-01
2020-03


### Price & Monthly Return Tables

In [287]:
price_prcd = pd.read_csv(path+'financial/close_price_processed_monthly.csv')
price_prcd.head()

,date,tic,adj_closed_price,monthly_return,forward_monthly_return,paper_month,month
0,2002-03-01,AAPL,1.47,0.088889,-0.117647,2002-02,2002-03
1,2002-04-01,AAPL,1.50,0.020408,0.088889,2002-03,2002-04
2,2002-05-01,AAPL,1.44,-0.040000,0.020408,2002-04,2002-05
3,2002-06-01,AAPL,1.10,-0.236111,-0.040000,2002-05,2002-06
4,2002-07-01,AAPL,0.95,-0.136364,-0.236111,2002-06,2002-07


In [238]:
startdate = '1990-01-01'
enddate = '2020-03-31'

# download price data
stockdata = pdr.get_data_yahoo(scholar_data_ticker_list, start=startdate, end=enddate)
closeadj = stockdata['Adj Close'].fillna(method='ffill').fillna(method='bfill').reset_index()

# calculate monthly return
closeadj['month'] = closeadj['Date'].apply(lambda x:str(x)[:7])
closeadj_month = closeadj.drop_duplicates(['month'],keep='last').set_index(['Date','month'])
monthly_return = (closeadj_month-closeadj_month.shift(1))/closeadj_month.shift(1)*100
monthly_return = monthly_return.drop(monthly_return.index[0]).fillna(0).reset_index()

# merge price and return data
price_prcd = pd.melt(closeadj_month.reset_index(), id_vars=['Date','month'], value_vars=scholar_data_ticker_list)
price_prcd.columns = ['date','month','tic','adj_closed_price']
return_prcd = pd.melt(monthly_return, id_vars=['Date','month'], value_vars=scholar_data_ticker_list)
return_prcd.columns = ['date','month','tic','monthly_return']
return_prcd['forward_monthly_return'] = return_prcd['monthly_return'].shift(1)
price_prcd = pd.merge(price_prcd,return_prcd,on=['date','month','tic'])
price_prcd = price_prcd.drop(price_prcd.index[0]).reset_index(drop=True)

# lag 6 months of scholar data 
paperdate = price_prcd['date'] - pd.DateOffset(months=1)
price_prcd['paper_month'] = paperdate.apply(lambda x:str(x)[:7])


In [239]:
price_prcd.head()

,date,month,tic,adj_closed_price,monthly_return,forward_monthly_return,paper_month
0,1990-03-30,1990-03,AAPL,1.166358,18.382361,0.322324,1990-02
1,1990-04-30,1990-04,AAPL,1.141003,-2.173910,18.382361,1990-03
2,1990-05-31,1990-05,AAPL,1.198654,5.052718,-2.173910,1990-04
3,1990-06-29,1990-06,AAPL,1.300358,8.484869,5.052718,1990-05
4,1990-07-31,1990-07,AAPL,1.220448,-6.145232,8.484869,1990-06


In [240]:
print(price_prcd.month.min())
print(price_prcd.month.max())

1990-02
2020-03


### Merge Fundamental, Price & Ratio Tables

In [275]:
financial_feature = pd.merge(funda_prcd, ratio_prcd, on=['month','tic'])
financial_feature = financial_feature.ffill()

financial_feature = pd.merge(financial_feature, price_prcd, on=['month','tic']).drop_duplicates(['month','tic'],keep='first')

In [276]:
print(financial_feature.month.min())
print(financial_feature.month.max())

1990-02
2019-01


In [277]:
financial_feature.head()

,month,tic,gsector,epspiq,prccq,pe_inc,ptb,ps,roa,roe,quick_ratio,curr_ratio,npm,de_ratio,date,adj_closed_price,monthly_return,forward_monthly_return,paper_month
0,1995-10,AVX,45,0.39,26.500,30.822,5.333,2.585,0.287,0.056,1.209,2.256,0.082,0.494,1995-10-31,9.572515,-6.790130,5.511842,1995-09
1,1995-11,AVX,45,0.39,26.500,23.566,4.296,2.263,0.316,0.129,1.317,2.382,0.094,0.472,1995-11-30,8.842086,-7.630479,-6.790130,1995-10
2,1995-12,AVX,45,0.39,26.500,21.721,3.960,2.086,0.316,0.129,1.317,2.382,0.094,0.472,1995-12-29,8.150093,-7.826126,-7.630479,1995-11
3,1996-01,AVX,45,0.42,21.875,21.516,3.922,2.067,0.316,0.129,1.317,2.382,0.094,0.472,1996-01-31,8.138925,-0.137035,-7.826126,1995-12
4,1996-02,AVX,45,0.42,21.875,17.626,3.504,1.829,0.337,0.189,1.360,2.456,0.103,0.456,1996-02-29,7.596328,-6.666683,-0.137035,1996-01


In [278]:
#financial_feature.to_csv(path+'financial/financial_features.csv', index=0)
financial_feature.to_csv(path+'financial/financial_features.csv', index=0)

### Merge Financial & Scholar Features

In [279]:
scholar_feature = pd.read_csv(path+'scholar_data/scholar_features.csv')

In [280]:
scholar_feature.shape

(9128, 50)

In [281]:
financial_feature.shape

(21600, 19)

In [282]:
# merge two tables while fill monthly scholar data with 0 and cumulatitive data with previous valid input
all_feature = pd.merge(financial_feature, scholar_feature, how='left',
                       left_on=['paper_month','tic'], right_on=['paper_month','ticker'])
all_feature = all_feature.drop(columns=['ticker'])

monthly_col=['total_count','total_citation','journal_count','journal_citation','patent_count','patent_citation',
             'article_count','article_citation','conference_count','conference_citation','book_count','book_citation',
             'total_max_citation','total_average_citation','patent_max_citation','patent_average_citation',
             'journal_max_citation','journal_average_citation','article_max_citation','article_average_citation',
             'conference_max_citation','conference_average_citation','book_max_citation','book_average_citation',
             'total_count_relative','journal_count_relative','patent_count_relative','article_count_relative',
             'conference_count_relative','book_count_relative']
cum_col = ['total_count_cumsum','total_citation_cumsum','journal_count_cumsum','journal_citation_cumsum','patent_count_cumsum',
           'patent_citation_cumsum','article_count_cumsum','article_citation_cumsum','conference_count_cumsum',
           'conference_citation_cumsum','book_count_cumsum','book_citation_cumsum','total_count_relative_cumsum',
           'journal_count_relative_cumsum','patent_count_relative_cumsum','article_count_relative_cumsum',
           'conference_count_relative_cumsum','book_count_relative_cumsum']

for col in monthly_col:
    all_feature[col] = all_feature[col].fillna(0)
for col in cum_col:
    all_feature[col] = all_feature[col].ffill().fillna(0)

all_feature = all_feature.drop_duplicates(['month','tic'],keep='first')


### Output Feature Table

In [286]:
#all_feature.to_csv(path+'result/fin+paper_feature_table.csv',index=0)
all_feature.to_csv(path+'result/fin+paper_feature_table.csv',index=0)


In [287]:
print(all_feature.tic.nunique())
print(all_feature.month.nunique())
print(max(all_feature.month))
print(min(all_feature.month))

86
348
2019-01
1990-02


In [288]:
all_feature.shape

(21600, 67)